In [2]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
import urllib.request
import os
from pyquery import PyQuery as pq
import pickle
from selenium import webdriver
import datetime as dt
import time
import calendar

In [3]:
#整理資料 將2016-01-01前的資料取出
origin_list=pd.read_csv("Materials Equity ETF List (62).csv",index_col=22,parse_dates=True)
origin_list2=pd.read_csv("Telecom Equity ETF List (10).csv",index_col=22,parse_dates=True)
origin_list=pd.concat([origin_list,origin_list2])
need_list=origin_list.sort_index().loc[:"2016-01-01"]
need_list.to_excel("new_list.xlsx")

In [4]:
#切出symbol欄&ETF名稱欄
need_list=pd.read_excel("new_list.xlsx")
symbol_list = need_list['Symbol'].reset_index()['Symbol']
name_list = need_list['ETF Name'].reset_index()['ETF Name']
all_ETF=need_list["Symbol"].values

In [5]:
#整理ETF Homepage 網站
homepage_list = []
for i in symbol_list:
    url = 'https://www.etf.com/'+i+'#overview'
    res = requests.get(url)
    doc = pq(res.text)
    doc.make_links_absolute(res.url)
    homepage_list.append(doc('.field-content.helplink')('a').attr('href'))
dic = {'symbol':[],'name':[],'addr':[]}
for i,s in enumerate(symbol_list):
    dic['symbol'].append(s)
    dic['name'].append(name_list[i])
    dic['addr'].append(homepage_list[i])
    print(s,name_list[i],homepage_list[i])
    
 # Save this dic
with open("ETF_dic.pkl","wb") as f:
    pickle.dump(dic,f)

XLB Materials Select Sector SPDR ETF https://us.spdrs.com/en/etf/materials-select-sector-spdr-fund-XLB
IYZ iShares U.S. Telecommunications ETF https://www.ishares.com/us/products/239523/ishares-us-telecommunications-etf
IYM iShares U.S. Basic Materials ETF https://www.ishares.com/us/products/239503/ishares-us-basic-materials-etf
IGE iShares North American Natural Resources ETF https://www.ishares.com/us/products/239768/ishares-north-american-natural-resources-etf
IXP iShares Global Telecom ETF https://www.ishares.com/us/products/239751/ishares-global-telecom-etf
VAW Vanguard Materials ETF https://personal.vanguard.com/us/funds/snapshot?FundId=0952&FundIntExt=INT
VOX Vanguard Communication Services ETF https://personal.vanguard.com/us/funds/snapshot?FundId=0959&FundIntExt=INT
PKB Invesco Dynamic Building & Construction ETF https://www.invesco.com/portal/site/us/financial-professional/etfs/product-detail?productId=pkb
XHB SPDR S&P Homebuilders ETF https://us.spdrs.com/en/etf/spdr-sp-home

In [6]:
with open('ETF_dic.pkl','rb') as f:
    dic = pickle.load(f)

In [7]:
#確認資料可從homepage的utl載下來，其他用yahoo finance的資料
#部分網站無法下載資料/無Nav, price
#只有Nav: spdrs, ishare,  vanguard, invesco, vaneck, nylinvestments, proshares,
yahoo_ETF = []
for i in range(len(all_ETF)):
    try:
        if "screener" in homepage_list[i]:
            continue
        elif "globalxfunds" in homepage_list[i]:
            continue
        else:
            yahoo_ETF.append(all_ETF[i])
    except:
        yahoo_ETF.append(all_ETF[i])

In [7]:
"""     if "spdrs" in homepage_list[i]: #spdrs網站資料
            continue
        elif "ishare" in homepage_list[i]: #ishare
            continue
        elif "vanguard" in homepage_list[i]: #vanguard
            continue
        elif "invesco" in homepage_list[i]: #invesco
            continue
        elif "vaneck" in homepage_list[i]: #vaneck
            continue
        elif "nylinvestments" in homepage_list[i]: #nylinvestments
            continue
        elif "screener" in homepage_list[i]: #screener
            continue
        elif "proshares" in homepage_list[i]: #proshares
            continue
        elif "globalxfunds" in homepage_list[i]: #globalxfunds
            continue
        else:
            yahoo_ETF.append(all_ETF[i])
    except:
        yahoo_ETF.append(all_ETF[i])
"""

'     if "spdrs" in homepage_list[i]: #spdrs網站資料\n            continue\n        elif "ishare" in homepage_list[i]: #ishare\n            continue\n        elif "vanguard" in homepage_list[i]: #vanguard\n            continue\n        elif "invesco" in homepage_list[i]: #invesco\n            continue\n        elif "vaneck" in homepage_list[i]: #vaneck\n            continue\n        elif "nylinvestments" in homepage_list[i]: #nylinvestments\n            continue\n        elif "screener" in homepage_list[i]: #screener\n            continue\n        elif "proshares" in homepage_list[i]: #proshares\n            continue\n        elif "globalxfunds" in homepage_list[i]: #globalxfunds\n            continue\n        else:\n            yahoo_ETF.append(all_ETF[i])\n    except:\n        yahoo_ETF.append(all_ETF[i])\n'

In [8]:
#timestamp now
#download ETFs at yahoo finance Web
Now = int(time.time())
for i in range(len(yahoo_ETF)):
    try:
        #目標網頁的網址
        url1 = "https://finance.yahoo.com/quote/"
        etf = yahoo_ETF[i]
        url2 = "/history?period1=1451577600&period2="
        now = str(Now)
        url3 = "&interval=1d&filter=history&frequency=1d"
        url = url1+etf+url2+now+url3
        #用chromedriver開啟Chrome
        options = webdriver.ChromeOptions()
        prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': './data'}
        options.add_experimental_option('prefs', prefs)
        driver = webdriver.Chrome(chrome_options=options)
        driver.get(url)

        #在網頁中找尋"Download Data並點擊"
        driver.find_element_by_link_text("Download Data").click()
        time.sleep(13)
        driver.quit()
        print("OK")
    except:
        print(yahoo_ETF[i])

/Users/jyun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


XLB


/Users/jyun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


OK


/Users/jyun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


...

OK


/Users/jyun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


OK


/Users/jyun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


OK


In [48]:
def download_read_screener(ETF_name):
    file_url= "https://screener.fidelity.com/ftgw/etf/downloadCSV.jhtml?symbol="+ETF_name
    file_name="data/" + ETF_name + ".csv"
    urllib.request.urlretrieve(file_url, file_name)
    data=pd.read_csv(file_name,index_col=0,parse_dates=True)
    ETF_list[ETF_name]=data["12/31/2015":"03/21/2019"]["Close"].values
    ETF_list[ETF_name].colums=[ETF_name]


def download_read_globalxfunds(ETF_name):
    file_url="https://www.globalxfunds.com/funds/" + ETF_name + "/?download_chart_data=true"
    file_name="data/" + ETF_name + ".csv"
    urllib.request.urlretrieve(file_url, file_name)
    data=pd.read_csv(file_name,index_col=0,header=1)
    data=data[:"12/31/15"][::-1]
    ETF_list[ETF_name]=data["Closing Price"].values
    ETF_list[ETF_name].colums=[ETF_name]


def read_yahoofin(ETF_name):
    file_name = './data/'+ETF_name+'.csv'
    ETF_list[ETF_name] = pd.read_csv(file_name,usecols = ['Adj Close'], parse_dates=True)
    ETF_list[ETF_name].columns=[ETF_name]

In [55]:
ETF_list = pd.read_csv("./data/DUST.csv",usecols=['Date'],parse_dates=True)
for i in range(len(all_ETF)):
    try:
        if  all_ETF[i] in yahoo_ETF:
            read_yahoofin(all_ETF[i])
        elif "screener" in homepage_list[i]:
            download_read_screener(all_ETF[i])
        elif "globalxfunds" in homepage_list[i]:
            download_read_globalxfunds(all_ETF[i])
    except:
        print(all_ETF[i]+"  No data!")    

print("done")
ETF_list.set_index(keys= ["Date"],inplace=True)

XLB  No data!
FTRI  No data!
done


In [56]:
ETF_list

,IYZ,IYM,IGE,IXP,VAW,VOX,PKB,XHB,ITB,GDX,...,FCOM,FMAT,SGDM,WBIF,GDXS,GDXX,HOML,SGDJ,NAIL,NANR
Date,,,,,,,,,,,,,,,,,,,,,
2015-12-31,26.884640,67.084175,26.567087,51.635830,89.324783,76.683502,23.566048,33.236561,26.761559,13.499532,...,26.7200,24.2809,12.658942,22.187315,108.400002,20.731684,22.620001,18.854044,28.391155,23.023586
2016-01-04,26.277653,65.971352,26.529324,50.948547,87.930702,75.595978,23.021452,32.604500,25.961672,13.863585,...,26.3199,23.8142,12.977158,21.773418,102.239998,22.050518,20.480000,19.569029,24.609661,22.996103
2016-01-05,26.501774,65.923790,26.585968,51.135986,87.892784,76.400192,22.991747,32.526711,26.030796,13.794711,...,26.5405,23.7700,12.912519,21.880295,103.480003,21.850695,20.690001,19.435825,25.088852,22.959452
2016-01-06,26.277653,64.154655,25.556896,50.734325,85.740028,76.071198,22.585775,31.729345,25.428415,14.021014,...,26.4400,23.2299,13.280454,21.724834,99.919998,22.709934,19.530001,20.176275,23.557011,22.318129
2016-01-07,25.614641,62.271393,24.943232,50.136292,83.331230,74.508461,21.971872,30.737497,24.371778,14.640891,...,25.9140,22.5700,13.911910,21.392553,92.000000,24.358479,17.351000,20.901054,20.483503,22.116570
2016-01-08,25.390528,61.681686,24.659998,49.948853,82.420830,73.923576,21.526295,30.202677,23.828646,14.286675,...,25.5864,22.3100,13.583753,21.156454,96.040001,23.339380,16.200001,20.146891,19.496643,21.777582
2016-01-11,25.418541,60.568840,24.018007,50.163074,81.263847,74.270844,21.437180,30.085987,23.966898,13.696318,...,25.7660,22.0380,12.977158,21.248758,103.199997,21.471029,16.520000,19.452477,19.344439,20.980509
2016-01-12,25.259790,60.673470,23.942478,50.296959,81.368156,73.841339,21.625313,30.358257,24.233528,13.391298,...,25.5301,22.0599,12.569446,21.423639,107.919998,20.541851,16.672001,19.098902,17.537649,20.723978
2016-01-13,24.568764,59.265789,23.347694,49.618603,79.499901,72.324287,20.912388,29.317795,23.512644,13.391298,...,25.0800,21.5300,12.658942,21.150623,108.040001,20.352020,15.810000,19.051888,18.912380,20.485773
